# Data Processing 

### Processsing avec Datasets

In [1]:
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.132:4041
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1608749071575)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession


In [2]:
val spark = SparkSession.
    builder.
    config("spark.ui.port", "0").
    appName("Data Processing").
    master("local[8]").
    getOrCreate

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@27e9a7f6


In [3]:
case class Person(id: Integer, firstName: String, middleName: String, lastName: String, gender: String, birthDate: String, ssn: String, salary: String)

defined class Person


In [4]:
val personDS = spark
  .read
  .option("header", "true")
  .option("inferSchema", "true")
  .option("delimiter", ",")
  .csv("datasets/people-with-header-10m.csv")
  .as[Person]

personDS: org.apache.spark.sql.Dataset[Person] = [id: int, firstName: string ... 6 more fields]


In [17]:
personDS

res8: org.apache.spark.sql.Dataset[Person] = [id: int, firstName: string ... 6 more fields]


In [18]:
personDS.show(5)

+---+---------+----------+--------+------+---------+-----------+--------+
| id|firstName|middleName|lastName|gender|birthDate|        ssn|  salary|
+---+---------+----------+--------+------+---------+-----------+--------+
|  1|  Fanchon|  Georgeta|   Wylma|Female|5/20/2020|537-80-6230|81273.63|
|  2|   Sandor|    Bordie| Humbert|  Male|3/12/2020|640-38-2361|11516.93|
|  3|  Quillan|       Ara| Hillery|  Male| 2/3/2020|647-45-5964|37622.42|
|  4|   Tallie|   Artemis|  Urbain|  Male|2/13/2020|714-11-1463|63124.83|
|  5|Katharine|    Elicia|   Muire|Female|2/28/2020|869-39-9071|44189.54|
+---+---------+----------+--------+------+---------+-----------+--------+
only showing top 5 rows



In [5]:
// DataFrame Api
println(personDS.filter($"firstName" === "Joe").distinct().count)

4


In [6]:
import org.apache.spark.sql.functions.col

import org.apache.spark.sql.functions.col


In [9]:
// Count le nombre de Personne dont le prenom est Joe
println(personDS.filter(col("firstName") === "Joe").distinct().count)

4


In [8]:
//F
println(personDS.filter(x => x.firstName == "Joe").distinct().count)

4


In [16]:
import org.apache.spark.sql.functions._
// define the year 40 years ago for the below query
import java.util.Calendar
val earliestYear = Calendar.getInstance.get(Calendar.YEAR) - 40

personDS
  .filter(year($"birthDate") > earliestYear) // everyone above 40
  .filter($"salary" > 1000) // everyone earning more than 80K
  .filter($"lastName".startsWith("J")) // last name starts with J
  .filter($"firstName".startsWith("M")) // first name starts with D
  .count()

import org.apache.spark.sql.functions._
import java.util.Calendar
earliestYear: Int = 1980
res9: Long = 0


## Preprocessing avec des données tabulaires

In [17]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder()
  .appName("Preprocesing column data")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()

// For implicit conversions like converting RDDs to DataFrames
import spark.implicits._

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@27e9a7f6
import spark.implicits._


### Creation d'un DataFrame

In [18]:
val l = List("X")

l: List[String] = List(X)


In [19]:
val df = l.toDF("Vars")

df: org.apache.spark.sql.DataFrame = [Vars: string]


In [20]:
df.printSchema

root
 |-- Vars: string (nullable = true)



In [21]:
df.show

+----+
|Vars|
+----+
|   X|
+----+



Une fois le DataFrame créé, nous pouvons l'utiliser pour comprendre comment utiliser les fonctions. Par exemple, pour obtenir la date actuelle, nous pouvons exécuter `df.select (current_date ()).show()`.

In [22]:
import org.apache.spark.sql.functions.current_date

import org.apache.spark.sql.functions.current_date


In [23]:
df.select(current_date).show

+--------------+
|current_date()|
+--------------+
|    2020-12-23|
+--------------+



In [24]:
val employees = List((1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                    )

employees: List[(Int, String, String, Double, String, String, String)] = List((1,Scott,Tiger,1000.0,united states,+1 123 456 7890,123 45 6789), (2,Henry,Ford,1250.0,India,+91 234 567 8901,456 78 9123), (3,Nick,Junior,750.0,united KINGDOM,+44 111 111 1111,222 33 4444), (4,Bill,Gomes,1500.0,AUSTRALIA,+61 987 654 3210,789 12 6118))


In [25]:
employees.size

res13: Int = 4


In [26]:
val employeesDF = employees.
    toDF("employee_id", "first_name",
         "last_name", "salary",
         "nationality", "phone_number",
         "ssn"
        )


employeesDF: org.apache.spark.sql.DataFrame = [employee_id: int, first_name: string ... 5 more fields]


In [27]:
employeesDF.printSchema

root
 |-- employee_id: integer (nullable = false)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- salary: double (nullable = false)
 |-- nationality: string (nullable = true)
 |-- phone_number: string (nullable = true)
 |-- ssn: string (nullable = true)



In [28]:
employeesDF.show(false)

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|nationality   |phone_number    |ssn        |
+-----------+----------+---------+------+--------------+----------------+-----------+
|1          |Scott     |Tiger    |1000.0|united states |+1 123 456 7890 |123 45 6789|
|2          |Henry     |Ford     |1250.0|India         |+91 234 567 8901|456 78 9123|
|3          |Nick      |Junior   |750.0 |united KINGDOM|+44 111 111 1111|222 33 4444|
|4          |Bill      |Gomes    |1500.0|AUSTRALIA     |+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



### Utilisation des fonctions Spark

In [29]:
// Pour l'utilisation de $ dans les functions à la place de col
import spark.implicits._

import spark.implicits._


In [30]:
employeesDF.
    select($"first_name", $"last_name").
    show

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
+----------+---------+



In [31]:
// Pour l'utilisation de col
import org.apache.spark.sql.functions.col

import org.apache.spark.sql.functions.col


In [32]:
// Utilisation de col 

employeesDF.
    select(col("first_name"), $"last_name").
    show

+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
+----------+---------+



In [33]:
// Utilisation du nom des colonnes.
employeesDF.
    select("first_name", "last_name").
    show



+----------+---------+
|first_name|last_name|
+----------+---------+
|     Scott|    Tiger|
|     Henry|     Ford|
|      Nick|   Junior|
|      Bill|    Gomes|
+----------+---------+



### groupBy colonne

In [34]:
employeesDF.
    groupBy("nationality").
    count.
    show

+--------------+-----+
|   nationality|count|
+--------------+-----+
|         India|    1|
|united KINGDOM|    1|
| united states|    1|
|     AUSTRALIA|    1|
+--------------+-----+



In [35]:
import org.apache.spark.sql.functions.upper

import org.apache.spark.sql.functions.upper


In [36]:
employeesDF.
    groupBy(upper($"nationality")).
    count.
    show

+------------------+-----+
|upper(nationality)|count|
+------------------+-----+
|    UNITED KINGDOM|    1|
|             INDIA|    1|
|         AUSTRALIA|    1|
|     UNITED STATES|    1|
+------------------+-----+



### orderBy colonne

In [37]:
employeesDF.
    orderBy("employee_id").
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+--------------+----------------+-----------+



Cependant, si nous voulons appliquer une transformation à l'aide de fonctions, passer des noms de colonnes sous forme de chaînes à certaines des fonctions ne suffira pas. Nous devons les passer comme type de colonne.

In [38]:
employeesDF.
    select(upper("first_name")).
    show

<console>: 69: error: type mismatch;

In [39]:
import org.apache.spark.sql.functions.upper

import org.apache.spark.sql.functions.upper


In [40]:
// version correcte
employeesDF.
    select(upper(col("first_name"))).
    show

+-----------------+
|upper(first_name)|
+-----------------+
|            SCOTT|
|            HENRY|
|             NICK|
|             BILL|
+-----------------+



In [41]:
// version alternative
employeesDF.
    select(upper($"first_name")).
    show

+-----------------+
|upper(first_name)|
+-----------------+
|            SCOTT|
|            HENRY|
|             NICK|
|             BILL|
+-----------------+



In [42]:
// dans le cas de groupBy
employeesDF.
    groupBy(upper($"nationality")).
    count.
    show

+------------------+-----+
|upper(nationality)|count|
+------------------+-----+
|    UNITED KINGDOM|    1|
|             INDIA|    1|
|         AUSTRALIA|    1|
|     UNITED STATES|    1|
+------------------+-----+



In [43]:
// Dans le cas de ordeBy
employeesDF.
    orderBy(upper($"nationality")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+--------------+----------------+-----------+



In [44]:
// version alternative
employeesDF.
    orderBy(upper(employeesDF("nationality"))).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+--------------+----------------+-----------+
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+--------------+----------------+-----------+



### Concatenation

In [45]:
import org.apache.spark.sql.functions.concat

import org.apache.spark.sql.functions.concat


Versions incorrectes de concatenation

In [46]:
employeesDF.
    select(concat($"first_name", ", ", $"last_name")).
    show()

<console>: 72: error: type mismatch;

In [47]:
// Deme
employeesDF.
    select(concat(col("first_name"), ", ", col("last_name"))).
    show

<console>: 73: error: type mismatch;

In [48]:
employeesDF.
    select(concat(employeesDF("first_name"), ", ", employeesDF("last_name"))).
    show

<console>: 72: error: type mismatch;

In [49]:
employeesDF.
    select(concat("first_name", ", ", "last_name")).
    show


<console>: 72: error: type mismatch;

versions correctes

In [50]:
import org.apache.spark.sql.functions.{concat, col, lit}

import org.apache.spark.sql.functions.{concat, col, lit}


In [51]:
employeesDF.
    select(concat(col("first_name"), lit(", "), col("last_name"))).
    show

+---------------------------------+
|concat(first_name, , , last_name)|
+---------------------------------+
|                     Scott, Tiger|
|                      Henry, Ford|
|                     Nick, Junior|
|                      Bill, Gomes|
+---------------------------------+



In [52]:
employeesDF.
    select(concat($"first_name", lit(", "), employeesDF("last_name"))).
    show

+---------------------------------+
|concat(first_name, , , last_name)|
+---------------------------------+
|                     Scott, Tiger|
|                      Henry, Ford|
|                     Nick, Junior|
|                      Bill, Gomes|
+---------------------------------+



### Manipulation de chaines de caractères

In [53]:
val employees = List((1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                    )


employees: List[(Int, String, String, Double, String, String, String)] = List((1,Scott,Tiger,1000.0,united states,+1 123 456 7890,123 45 6789), (2,Henry,Ford,1250.0,India,+91 234 567 8901,456 78 9123), (3,Nick,Junior,750.0,united KINGDOM,+44 111 111 1111,222 33 4444), (4,Bill,Gomes,1500.0,AUSTRALIA,+61 987 654 3210,789 12 6118))


In [54]:
val employeesDF = employees.
    toDF("employee_id", "first_name",
         "last_name", "salary",
         "nationality", "phone_number",
         "ssn"
        )

employeesDF: org.apache.spark.sql.DataFrame = [employee_id: int, first_name: string ... 5 more fields]


In [55]:
import org.apache.spark.sql.functions.{col, upper, lower, initcap, length}

import org.apache.spark.sql.functions.{col, upper, lower, initcap, length}


In [56]:
employeesDF.
    select("employee_id", "nationality").
    withColumn("nationality_upper", upper(col("nationality"))).
    withColumn("nationality_lower", lower($"nationality")).
    withColumn("nationality_initcap", initcap(employeesDF("nationality"))).
    withColumn("nationality_length", length(col("nationality"))).
    show

+-----------+--------------+-----------------+-----------------+-------------------+------------------+
|employee_id|   nationality|nationality_upper|nationality_lower|nationality_initcap|nationality_length|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+
|          1| united states|    UNITED STATES|    united states|      United States|                13|
|          2|         India|            INDIA|            india|              India|                 5|
|          3|united KINGDOM|   UNITED KINGDOM|   united kingdom|     United Kingdom|                14|
|          4|     AUSTRALIA|        AUSTRALIA|        australia|          Australia|                 9|
+-----------+--------------+-----------------+-----------------+-------------------+------------------+



**Substring**

In [57]:
import org.apache.spark.sql.functions.substring

import org.apache.spark.sql.functions.substring


In [58]:
import spark.implicits._

import spark.implicits._


In [59]:
employeesDF.
    select("employee_id", "phone_number", "ssn").
    withColumn("phone_last4", substring($"phone_number", -4, 4).cast("int")).
    withColumn("ssn_last4", substring($"ssn", 8, 4).cast("int")).
    show

+-----------+----------------+-----------+-----------+---------+
|employee_id|    phone_number|        ssn|phone_last4|ssn_last4|
+-----------+----------------+-----------+-----------+---------+
|          1| +1 123 456 7890|123 45 6789|       7890|     6789|
|          2|+91 234 567 8901|456 78 9123|       8901|     9123|
|          3|+44 111 111 1111|222 33 4444|       1111|     4444|
|          4|+61 987 654 3210|789 12 6118|       3210|     6118|
+-----------+----------------+-----------+-----------+---------+



**Splitting**

In [60]:
val employees = List((1, "Scott", "Tiger", 1000.0, 
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, 
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, 
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                    )

employees: List[(Int, String, String, Double, String, String, String)] = List((1,Scott,Tiger,1000.0,united states,+1 123 456 7890,123 45 6789), (2,Henry,Ford,1250.0,India,+91 234 567 8901,456 78 9123), (3,Nick,Junior,750.0,united KINGDOM,+44 111 111 1111,222 33 4444), (4,Bill,Gomes,1500.0,AUSTRALIA,+61 987 654 3210,789 12 6118))


In [61]:
val employeesDF = employees.
    toDF("employee_id", "first_name",
         "last_name", "salary",
         "nationality", "phone_number",
         "ssn"
        )

employeesDF: org.apache.spark.sql.DataFrame = [employee_id: int, first_name: string ... 5 more fields]


In [62]:
import org.apache.spark.sql.functions.split

import org.apache.spark.sql.functions.split


In [63]:
import spark.implicits._

import spark.implicits._


In [64]:
employeesDF.
    select("employee_id", "phone_number", "ssn").
    withColumn("area_code", split($"phone_number", " ")(1).cast("int")).
    withColumn("phone_last4", split($"phone_number", " ")(3).cast("int")).
    withColumn("ssn_last4", split($"ssn", " ")(2).cast("int")).
    show

+-----------+----------------+-----------+---------+-----------+---------+
|employee_id|    phone_number|        ssn|area_code|phone_last4|ssn_last4|
+-----------+----------------+-----------+---------+-----------+---------+
|          1| +1 123 456 7890|123 45 6789|      123|       7890|     6789|
|          2|+91 234 567 8901|456 78 9123|      234|       8901|     9123|
|          3|+44 111 111 1111|222 33 4444|      111|       1111|     4444|
|          4|+61 987 654 3210|789 12 6118|      987|       3210|     6118|
+-----------+----------------+-----------+---------+-----------+---------+



In [65]:
employeesDF.
    select($"employee_id", $"phone_number", $"ssn", 
           split($"phone_number", " ")(1).cast("int").alias("area_code"),
           split($"phone_number", " ")(3).cast("int").alias("phone_last"),
           split($"ssn", " ")(2).cast("int").alias("ssn_last4")
          ).
    show

+-----------+----------------+-----------+---------+----------+---------+
|employee_id|    phone_number|        ssn|area_code|phone_last|ssn_last4|
+-----------+----------------+-----------+---------+----------+---------+
|          1| +1 123 456 7890|123 45 6789|      123|      7890|     6789|
|          2|+91 234 567 8901|456 78 9123|      234|      8901|     9123|
|          3|+44 111 111 1111|222 33 4444|      111|      1111|     4444|
|          4|+61 987 654 3210|789 12 6118|      987|      3210|     6118|
+-----------+----------------+-----------+---------+----------+---------+



Creation d'une nouvelle colonne

In [66]:
import org.apache.spark.sql.functions.concat
import spark.implicits._

import org.apache.spark.sql.functions.concat
import spark.implicits._


In [67]:
employeesDF.
    withColumn("full_name", concat($"first_name", $"last_name")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+----------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn| full_name|
+-----------+----------+---------+------+--------------+----------------+-----------+----------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|ScottTiger|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123| HenryFord|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|NickJunior|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118| BillGomes|
+-----------+----------+---------+------+--------------+----------------+-----------+----------+



In [68]:
import org.apache.spark.sql.functions.{concat, lit}

import org.apache.spark.sql.functions.{concat, lit}


In [69]:
employeesDF.
    withColumn("full_name", concat($"first_name", lit(", "), $"last_name")).
    show

+-----------+----------+---------+------+--------------+----------------+-----------+------------+
|employee_id|first_name|last_name|salary|   nationality|    phone_number|        ssn|   full_name|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+
|          1|     Scott|    Tiger|1000.0| united states| +1 123 456 7890|123 45 6789|Scott, Tiger|
|          2|     Henry|     Ford|1250.0|         India|+91 234 567 8901|456 78 9123| Henry, Ford|
|          3|      Nick|   Junior| 750.0|united KINGDOM|+44 111 111 1111|222 33 4444|Nick, Junior|
|          4|      Bill|    Gomes|1500.0|     AUSTRALIA|+61 987 654 3210|789 12 6118| Bill, Gomes|
+-----------+----------+---------+------+--------------+----------------+-----------+------------+



### Padding

In [70]:
import org.apache.spark.sql.functions.{lit, lpad}

import org.apache.spark.sql.functions.{lit, lpad}


In [71]:
val l = List("X")

l: List[String] = List(X)


In [72]:
val df = l.toDF("dummy")

df: org.apache.spark.sql.DataFrame = [dummy: string]


In [73]:
df.select(lpad(lit("Hello"), 10, "-").alias("dummy")).show

+----------+
|     dummy|
+----------+
|-----Hello|
+----------+



Utilisez les fonctions de pad pour convertir chacun des champs en longueur fixe et concaténer. Voici les détails de chacun des champs.

* La longueur de employee_id doit être de 5 caractères et doit être complétée par zéro.
* La longueur de first_name et last_name doit être de 10 caractères et doit être complétée par - sur le côté droit.
* La longueur du salaire doit être de 10 caractères et doit être complétée par zéro.
*  La longueur de la nationalité doit être de 15 caractères et doit être complétée par - sur le côté droit.
* La longueur du phone_number doit être de 17 caractères et doit être complétée par - sur le côté droit.
* La longueur du ssn peut être laissée telle quelle. C'est 11 caractères.

In [74]:
import org.apache.spark.sql.functions.{lpad, rpad, concat}

import org.apache.spark.sql.functions.{lpad, rpad, concat}


In [75]:
import spark.implicits._

import spark.implicits._


In [76]:
val empFixedDF = employeesDF.select(
    concat(
        lpad($"employee_id", 5, "0"),
        rpad($"first_name", 10, "-"),
        rpad($"last_name", 10, "-"),
        lpad($"salary", 10, "0"),
        rpad($"nationality", 15, "-"),
        rpad($"phone_number", 17, "-"),
        $"ssn"
    ).alias("employee")
)

empFixedDF: org.apache.spark.sql.DataFrame = [employee: string]


In [77]:
empFixedDF.show(false)

+------------------------------------------------------------------------------+
|employee                                                                      |
+------------------------------------------------------------------------------+
|00001Scott-----Tiger-----00001000.0united states--+1 123 456 7890--123 45 6789|
|00002Henry-----Ford------00001250.0India----------+91 234 567 8901-456 78 9123|
|00003Nick------Junior----00000750.0united KINGDOM-+44 111 111 1111-222 33 4444|
|00004Bill------Gomes-----00001500.0AUSTRALIA------+61 987 654 3210-789 12 6118|
+------------------------------------------------------------------------------+



**Trimming**

In [78]:
val l = List("   Hello.    ")

l: List[String] = List("   Hello.    ")


In [79]:
val df = l.toDF("dummy")

df: org.apache.spark.sql.DataFrame = [dummy: string]


In [80]:
import org.apache.spark.sql.functions.{col, ltrim, rtrim, trim}

import org.apache.spark.sql.functions.{col, ltrim, rtrim, trim}


In [81]:
df.withColumn("ltrim", ltrim(col("dummy"))).
    withColumn("rtrim", rtrim(rtrim(col("dummy")), ".")).
    withColumn("trim", trim(trim(col("dummy")), ".")).
    show()

+-------------+----------+--------+-----+
|        dummy|     ltrim|   rtrim| trim|
+-------------+----------+--------+-----+
|   Hello.    |Hello.    |   Hello|Hello|
+-------------+----------+--------+-----+



## Date et Time

In [82]:
val l = List("X")
val df = l.toDF("dummy")

l: List[String] = List(X)
df: org.apache.spark.sql.DataFrame = [dummy: string]


In [83]:
import org.apache.spark.sql.functions.{current_date, current_timestamp}

import org.apache.spark.sql.functions.{current_date, current_timestamp}


In [84]:
df.select(current_date.alias("current_date")).show

+------------+
|current_date|
+------------+
|  2020-12-23|
+------------+



In [85]:
df.select(current_timestamp.alias("current_time")).show(false)

+-------------------------+
|current_time             |
+-------------------------+
|2020-12-23 18:51:14.25466|
+-------------------------+



In [86]:
val datetimes = List(("2014-02-28", "2014-02-28 10:00:00.123"),
                     ("2016-02-29", "2016-02-29 08:08:08.999"),
                     ("2017-10-31", "2017-12-31 11:59:59.123"),
                     ("2019-11-30", "2019-08-31 00:00:00.000")
                    )

datetimes: List[(String, String)] = List((2014-02-28,2014-02-28 10:00:00.123), (2016-02-29,2016-02-29 08:08:08.999), (2017-10-31,2017-12-31 11:59:59.123), (2019-11-30,2019-08-31 00:00:00.000))


In [87]:
val datetimesDF = datetimes.toDF("date", "time")

datetimesDF: org.apache.spark.sql.DataFrame = [date: string, time: string]


In [88]:
datetimesDF.show(false)

+----------+-----------------------+
|date      |time                   |
+----------+-----------------------+
|2014-02-28|2014-02-28 10:00:00.123|
|2016-02-29|2016-02-29 08:08:08.999|
|2017-10-31|2017-12-31 11:59:59.123|
|2019-11-30|2019-08-31 00:00:00.000|
+----------+-----------------------+

